In [ ]:
import pandas as pd

lr_df = pd.read_csv("whisper-tiny-lr-only/summary.csv")
hr_df = pd.read_csv("whisper-tiny/summary.csv")
lr_df = lr_df.rename(columns={'wer': 'wer_orig'})
hr_df = hr_df.rename(columns={'wer': 'wer_hr'})

In [ ]:
merged = pd.merge(lr_df, hr_df, on='language', how='inner')


In [ ]:
merged['diff'] = merged.apply(lambda row: row.wer_orig - row.wer_hr, axis=1)
merged = merged.sort_values(by='diff', ascending=False)
merged

,language,wer_orig,wer_hr,diff
3,bas,0.442412,0.429450,0.012962
7,lth,0.442338,0.442338,0.000000
19,led,0.444518,0.444518,0.000000
22,ttj,0.497165,0.471581,0.025584
25,pne,0.506414,0.474234,0.032180
13,ukv,0.570485,0.545388,0.025097
2,meh,0.641875,0.622307,0.019569
11,ush,0.625301,0.623791,0.001510
9,sco,0.635369,0.635369,0.000000
10,cgg,0.723558,0.650350,0.073208
